# GPU Computing with Julia

In [1]:
gethostname()

"dgx-01"

## Topology of a GPU

<img src="../imgs/gpu_topology.png" width=1300px>

**Source:** [Sivalingam, Karthee. "GPU Acceleration of a Theoretical Particle Physics Application." Master's Thesis, The University of Edinburgh (2010).](https://static.epcc.ed.ac.uk/dissertations/hpc-msc/2009-2010/Karthee%20Sivalingam.pdf)

* **SM** = Streaming Multiprocessor
* **SP** = Streaming Processor

### NVIDIA A100 SXM4

<img src="../imgs/a100_front.png" width=800px>

<img src="../imgs/a100_SM.png" width=400px>

**Source:** [NVIDIA whitepaper](https://images.nvidia.com/aem-dam/en-zz/Solutions/data-center/nvidia-ampere-architecture-whitepaper.pdf)

| Kind                       | Count            |
|----------------------------|------------------|
| **SMs**                    | 108              |
| **CUDA cores** / FP32 ALUs | 6912 (64 per SM) |
| **Tensor cores**           | 432 (4 per SM)   |

* **ALU** = Arithmetic Logical Unit


In [10]:
# using GPUInspector
# gpuinfo()

## JuliaGPU

Website: https://juliagpu.org/

GitHub Org: https://github.com/JuliaGPU




(We'll focus on Nvidia GPUs but there is [support for other GPUs](https://juliagpu.org/) as well.)

The interface to NVIDIA GPU computing in Julia is [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl).



It provides:

* **High-level abstraction `CuArray`**
* **Tools for writing custom CUDA kernels**
* **Wrappers to proprietary NVIDIA libraries (e.g. CUBLAS, CUFFT, CUSPARSE)**

In [2]:
using CUDA

In [5]:
CUDA.versioninfo() # automatically downloads CUDA framework if necessary

CUDA toolkit 11.7, artifact installation
NVIDIA driver 510.47.3, for CUDA 11.6
CUDA driver 11.7

Libraries: 
- CUBLAS: 11.10.1
- CURAND: 10.2.10
- CUFFT: 10.7.2
- CUSOLVER: 11.3.5
- CUSPARSE: 11.7.3
- CUPTI: 17.0.0
- NVML: 11.0.0+510.47.3
- CUDNN: 8.30.2 (for CUDA 11.5.0)
- CUTENSOR: 1.4.0 (for CUDA 11.5.0)

Toolchain:
- Julia: 1.8.0
- LLVM: 13.0.1
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0, 7.1, 7.2
- Device capability support: sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80, sm_86

8 devices:
  0: NVIDIA A100-SXM4-40GB (sm_80, 39.406 GiB / 40.000 GiB available)
  1: NVIDIA A100-SXM4-40GB (sm_80, 39.406 GiB / 40.000 GiB available)
  2: NVIDIA A100-SXM4-40GB (sm_80, 39.406 GiB / 40.000 GiB available)
  3: NVIDIA A100-SXM4-40GB (sm_80, 39.406 GiB / 40.000 GiB available)
  4: NVIDIA A100-SXM4-40GB (sm_80, 39.406 GiB / 40.000 GiB available)
  5: NVIDIA A100-SXM4-40GB (sm_80, 39.406 GiB / 40.000 GiB available)
  6: NVIDI

In [4]:
CUDA.functional()

true

## High-level abstraction: `CuArray`

### GPU memory

A `CuArray` is a CPU handle to GPU memory.

In [6]:
x_gpu = CuArray{Float32}(undef, 3)

3-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.0
 0.0
 0.0

In [7]:
CUDA.rand(3) # Note: defaults to Float32

3-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.5646155
 0.5066599
 0.9901406

In [8]:
CUDA.zeros(3) # Note: defaults to Float32

3-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.0
 0.0
 0.0

 We can readily move data to the GPU by converting to `CuArray`.

 <img src="../imgs/cpu_gpu_transfer.svg" width=180px>

In [9]:
x_cpu = [1,2,3]
x_gpu = CuArray(x_cpu) 

3-element CuArray{Int64, 1, CUDA.Mem.DeviceBuffer}:
 1
 2
 3

(or by using `copyto!` to move it into already allocated memory)

#### Memory management

`CuArray`s are managed by Julia's **garbage collector**. In principle, if they are unreachable, they get cleaned up automatically.

However, by default CUDA.jl uses a **memory pool** to speed up allocations. So it might appear as if the objects have not been free'd. (You can disable the pool with `JULIA_CUDA_MEMORY_POOL=none`.)

In [10]:
CUDA.memory_status()

Effective GPU memory usage: 1.13% (455.938 MiB/39.409 GiB)
Memory pool usage: 36 bytes (32.000 MiB reserved)

In [33]:
x_gpu = CUDA.rand(10_000_000);

In [34]:
Base.format_bytes(sizeof(x_gpu))

"38.147 MiB"

In [35]:
CUDA.memory_status()

Effective GPU memory usage: 1.21% (487.938 MiB/39.409 GiB)
Memory pool usage: 38.147 MiB (64.000 MiB reserved)

In [36]:
x_gpu = nothing; GC.gc(true)

In [37]:
CUDA.memory_status()

Effective GPU memory usage: 1.21% (487.938 MiB/39.409 GiB)
Memory pool usage: 38.147 MiB (64.000 MiB reserved)

We can use `CUDA.unsafe_free!(x_gpu)` and `CUDA.reclaim()` to more aggressively suggest the freeing of the memory.

In [29]:
#CUDA.unsafe_free!(x_gpu)

In [32]:
#CUDA.reclaim()

### GPU computation

In [38]:
CuArray <: AbstractArray

true

Therefore, we should be able to do all kind of operations with it, that we'd also do with regular `Array`s. (**duck typing**)

#### Example: Matrix multiplication

In [40]:
N = 1000
A_gpu = CUDA.rand(N,N)
B_gpu = CUDA.rand(N,N)

1000×1000 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.69478    0.0289907  0.0183466  …  0.339742   0.0545384  0.965939
 0.817242   0.0547828  0.302184      0.717154   0.561394   0.473867
 0.622881   0.495846   0.0230539     0.0381533  0.583806   0.477023
 0.447436   0.803525   0.71222       0.395164   0.352538   0.309192
 0.964878   0.442542   0.370591      0.0926875  0.624995   0.847689
 0.665145   0.770985   0.0382734  …  0.533212   0.416791   0.626831
 0.483884   0.279765   0.484469      0.660593   0.520294   0.422101
 0.166863   0.525936   0.257448      0.308513   0.497302   0.289458
 0.647593   0.616969   0.548403      0.304596   0.581397   0.283971
 0.0499254  0.882947   0.675325      0.72163    0.532929   0.825365
 ⋮                                ⋱                        
 0.846527   0.383507   0.746302      0.18618    0.990434   0.119101
 0.956102   0.719037   0.424064      0.166237   0.179259   0.714286
 0.959444   0.0528611  0.528541      0.674956   0.61374    0.762448
 0

In [41]:
A_gpu * B_gpu

1000×1000 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 263.231  250.303  249.849  246.12   …  251.736  264.457  255.108  261.233
 266.702  262.424  256.225  257.277     262.914  270.869  258.223  262.323
 258.117  253.468  243.063  248.039     252.824  261.536  256.234  256.848
 255.43   250.546  248.836  246.455     252.775  262.942  255.662  256.531
 255.411  247.417  243.252  244.947     244.026  254.167  244.498  248.254
 253.226  242.999  246.033  241.936  …  247.346  254.422  242.234  251.317
 257.207  253.969  246.566  241.779     248.356  258.27   251.288  252.969
 257.595  252.352  248.305  246.535     250.534  257.406  255.271  255.107
 256.171  253.301  251.302  247.941     247.637  257.632  256.604  253.424
 252.13   247.081  244.887  240.462     246.285  259.547  252.714  250.765
   ⋮                                 ⋱                             
 264.99   264.5    263.187  253.443     260.07   269.754  261.162  268.605
 256.06   253.018  247.612  244.237     249.991  253.

In [45]:
using BenchmarkTools

@btime A_cpu * B_cpu setup=(A_cpu = rand(Float32, N,N); B_cpu = rand(Float32, N,N););
@btime A_gpu * B_gpu setup=(A_gpu = CUDA.rand(N,N); B_gpu = CUDA.rand(N,N););

  3.632 ms (2 allocations: 3.81 MiB)


  17.894 μs (29 allocations: 592 bytes)


Note how the timescales change: **milliseconds -> microseconds**!

(`*` for `CuArray`s uses a cuBLAS kernel under the hood)

#### Example: Broadcasting, `map`, `reduce`, etc.

In [46]:
A_gpu .+ B_gpu # runs on the GPU!

1000×1000 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.40741   0.19631   0.316908   0.438691  …  0.479184  0.841024  1.94615
 0.986012  0.747058  0.903933   0.811005     0.973116  1.40697   1.30586
 0.858665  1.24315   0.718586   0.793501     0.57865   0.81746   0.797016
 1.29973   1.03181   0.997126   1.21855      1.18421   0.629287  0.696938
 1.90947   0.891688  1.14663    0.593971     0.471222  0.696889  1.30617
 1.48993   1.64138   0.57461    0.207497  …  1.03745   0.440282  1.62009
 0.637022  0.685722  1.41646    0.950116     0.926303  0.94046   1.11532
 0.941927  0.530214  0.517546   0.317172     1.03441   1.3146    0.966244
 0.693739  0.851431  1.35988    0.840186     1.08617   0.685314  0.310763
 0.558172  1.37046   1.22602    0.76166      0.837085  1.21962   1.3164
 ⋮                                        ⋱                      
 1.29783   0.418178  0.803072   0.954879     0.999511  1.59043   0.516183
 1.49435   0.725889  1.06637    0.385113     1.02486   0.717913  0.791807

In [47]:
sqrt.(A_gpu.^2 + B_gpu.^2) # runs on the GPU!

1000×1000 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.995272  0.169813  0.299124   0.381416  …  0.367245  0.788375  1.37617
 0.834487  0.69444   0.673362   0.605327     0.761463  1.01497   0.95748
 0.666014  0.896846  0.695914   0.730457     0.541842  0.628827  0.57441
 0.962606  0.835325  0.767091   0.862003     0.88247   0.448188  0.49593
 1.35027   0.630536  0.859987   0.592642     0.389717  0.629117  0.963734
 1.05957   1.16276   0.537701   0.148388  …  0.733872  0.417452  1.17451
 0.507538  0.493021  1.05039    0.74004      0.712029  0.668764  0.811618
 0.792822  0.525953  0.365965   0.22832      0.788737  0.956706  0.736088
 0.649235  0.660018  0.979412   0.693964     0.838834  0.590611  0.285232
 0.510693  1.00859   0.871396   0.596788     0.730807  0.86923   0.960385
 ⋮                                        ⋱                      
 0.959315  0.385071  0.748458   0.705959     0.834368  1.15799   0.41456
 1.0972    0.719069  0.769663   0.289522     0.874567  0.567699  0.71848

In [48]:
mapreduce(sin, +, A_gpu) # runs on the GPU!

459606.56f0

#### "Counter-example:" Scalar indexing

In [49]:
A_gpu[1]

ErrorException: Scalar indexing is disallowed.
Invocation of getindex resulted in scalar indexing of a GPU array.
This is typically caused by calling an iterating implementation of a method.
Such implementations *do not* execute on the GPU, but very slowly on the CPU,
and therefore are only permitted from the REPL for prototyping purposes.
If you did intend to index this array, annotate the caller with @allowscalar.

In [50]:
CUDA.@allowscalar A_gpu[1]

0.7126331f0

In [52]:
function gpu_not_actually!(C, A, B)
    CUDA.@sync CUDA.@allowscalar for i in eachindex(A,B)
        C[i] = A[i] * B[i] # multiplication will happen on CPU!
    end
end

function gpu_broadcasting!(C, A, B)
    CUDA.@sync C .= A .* B
end

gpu_broadcasting! (generic function with 1 method)

In [54]:
using BenchmarkTools

N = 10
@btime gpu_not_actually!(C, A, B) setup=(A = CUDA.rand(10,10); B = CUDA.rand(10,10); C = CUDA.rand(10,10););
@btime gpu_broadcasting!(C, A, B) setup=(A = CUDA.rand(10,10); B = CUDA.rand(10,10); C = CUDA.rand(10,10););

  4.906 ms

 (601 allocations: 135.95 KiB)


  15.750 μs (24 allocations: 1.83 KiB)


##### FLoops: CUDA executor

In [57]:
using FLoops, FoldsCUDA

function gpu_floops!(C, A, B)
    CUDA.@sync @floop CUDAEx() for i in eachindex(A,B,C)
        C[i] = A[i] * B[i]
    end
end

gpu_floops! (generic function with 1 method)

In [58]:
@btime gpu_floops!(C, A, B) setup=(A = CUDA.rand(10,10); B = CUDA.rand(10,10); C = CUDA.rand(10,10););

  52.169 μs (237 allocations: 19.67 KiB)


## Kernel programming: Writing CUDA kernels

A CUDA kernel is a function that will be executed by all GPU *threads* separately. Based on the index of a thread we can make them operate on different pieces of given data (Single Program Multiple Data (SPMD) programming model similar to MPI).

In [59]:
x = CUDA.zeros(1024)

function cuda_kernel!(x)
    i = threadIdx().x
    x[i] += 1
    return nothing # CUDA kernels should never return anything
end

cuda_kernel! (generic function with 1 method)

In [60]:
CUDA.@sync @cuda threads=length(x) cuda_kernel!(x)

CUDA.HostKernel{typeof(cuda_kernel!), Tuple{CuDeviceVector{Float32, 1}}}(cuda_kernel!, CuFunction(Ptr{Nothing} @0x000000003e1674f0, CuModule(Ptr{Nothing} @0x000000003ec4e3e0, CuContext(0x0000000002caaa60, instance f2e7009325be14e3))), CUDA.KernelState(Ptr{Nothing} @0x00007f45aa400000))

In [61]:
x

1024-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

Kernel programming can quickly become (much) more difficult though because
* you need to respect **hardware limitations** of the GPU
* **not all operations can readily be expressed as scalar kernels** (e.g. reductions)
* since kernels execute on the GPU, the Julia runtime isn't available and kernel code has limitations (**you can't just write arbitrary Julia code in kernels**)
  * no GC / no allocations
  * must be fully type inferred
  * no `try ... catch ... end`
  * no strings
  * ...

Simple example for a hardware limitation: **A100 supports a maximal number of 1024 threads.**

In [68]:
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)

1024

In [62]:
x = CUDA.zeros(1025)
CUDA.@sync @cuda threads=length(x) cuda_kernel!(x)

CuError: CUDA error: invalid argument (code 1, ERROR_INVALID_VALUE)

**But what if we want to go larger?**

### CUDA programming model

<img src="../imgs/cuda_blocks_threads.png" width=700px>

(Note: in Julia indices start at 1)

**Source:** [Sivalingam, Karthee. "GPU Acceleration of a Theoretical Particle Physics Application." Master's Thesis, The University of Edinburgh (2010).](https://static.epcc.ed.ac.uk/dissertations/hpc-msc/2009-2010/Karthee%20Sivalingam.pdf)

* **Threads** → CUDA cores
* **Blocks** of threads → SMs
* **Grid** of blocks → entire GPU

In [63]:
function cuda_kernel_blocks!(x)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    if i <= length(x)
        @inbounds x[i] += 1
    end
    return nothing # CUDA kernels should never return anything
end

cuda_kernel_blocks! (generic function with 1 method)

In [64]:
x = CUDA.zeros(1025)
CUDA.@sync @cuda threads=1024 blocks=2 cuda_kernel_blocks!(x)

CUDA.HostKernel{typeof(cuda_kernel_blocks!), Tuple{CuDeviceVector{Float32, 1}}}(cuda_kernel_blocks!, CuFunction(Ptr{Nothing} @0x000000004112e5f0, CuModule(Ptr{Nothing} @0x000000003e861280, CuContext(0x0000000002caaa60, instance f2e7009325be14e3))), CUDA.KernelState(Ptr{Nothing} @0x00007f45aa400000))

In [65]:
x

1025-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

How does our custom CUDA kernel compare to broadcasting?

In [67]:
function add_one_kernel(x)
    CUDA.@sync @cuda threads=1024 blocks=1 cuda_kernel_blocks!(x)
end

function add_one_broadcasting(x)
    CUDA.@sync x .+ 1
end

@btime add_one_kernel(x) setup=(x = CUDA.zeros(1024););
@btime add_one_broadcasting(x) setup=(x = CUDA.zeros(1024););

  13.234 μs (5 allocations: 304 bytes)


  16.160 μs (30 allocations: 1.58 KiB)


### Simplifying kernel launches: [Occupancy API](https://developer.nvidia.com/blog/cuda-pro-tip-occupancy-api-simplifies-launch-configuration/)

Hardcoding limits (1024 above) is rarely a good idea. Also, in reality, the actual maximal number of threads can depend on kernel details, like how many resources the kernel is using.

**The occupancy API is an automatic tool that can be used to obtain good launch parameters.**

In [71]:
kernel = @cuda launch=false cuda_kernel_blocks!(x)

CUDA.HostKernel{typeof(cuda_kernel_blocks!), Tuple{CuDeviceVector{Float32, 1}}}(cuda_kernel_blocks!, CuFunction(Ptr{Nothing} @0x000000004112e5f0, CuModule(Ptr{Nothing} @0x000000003e861280, CuContext(0x0000000002caaa60, instance f2e7009325be14e3))), CUDA.KernelState(Ptr{Nothing} @0x00007f45aa400000))

In [72]:
CUDA.maxthreads(kernel)

1024

In [80]:
config = CUDA.launch_configuration(kernel.fun)

(blocks = 216, threads = 1024)

Here, the number `blocks` indicates how many blocks we would need to fully occupy the GPU. For a given input `x`, we might need fewer or more blocks.

In [82]:
threads = min(length(x), config.threads)

1024

In [83]:
blocks = cld(length(x), threads)

2

Launching the kernel with the dynamic launch parameters:

In [84]:
kernel(x; threads, blocks)

## Case study: Three ways to SAXPY on the GPU

**SAXPY** = **S**ingle precision **A** times **X** **P**lus **Y**

In [65]:
"Computes the SAXPY on the CPU using broadcasting"
function saxpy_broadcast_cpu!(a, x, y)
    y .= a .* x .+ y
end

saxpy_broadcast_cpu!

In [66]:
"Computes the SAXPY on the GPU using broadcasting"
function saxpy_broadcast_gpu!(a, x, y)
    CUDA.@sync y .= a .* x .+ y
end

saxpy_broadcast_gpu!

In [68]:
"CUDA kernel for computing the SAXPY on the GPU"
function _saxpy_kernel!(a, x, y)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    if i <= length(y)
        @inbounds y[i] = a * x[i] + y[i]
    end
    return nothing
end

"Computes the SAXPY on the GPU using the custom CUDA kernel `_saxpy_kernel!`"
function saxpy_cuda_kernel!(a, x, y; nthreads, nblocks)
    CUDA.@sync @cuda(
        threads = nthreads,
        blocks = nblocks,
        _saxpy_kernel!(a, x, y)
    )
end

saxpy_cuda_kernel!

In [70]:
function saxpy_cublas!(a, x, y)
    CUDA.@sync CUBLAS.axpy!(length(x), a, x, y)
end

saxpy_cublas! (generic function with 1 method)

In [73]:
using PrettyTables

"Computes the GFLOP/s from the vector length `len` and the measured runtime `t`."
saxpy_flops(t; len) = 2.0 * len * 1e-9 / t # GFLOP/s

"Computes the GB/s from the vector length `len`, the vector element type `dtype`, and the measured runtime `t`."
saxpy_bandwidth(t; dtype, len) = 3.0 * sizeof(dtype) * len * 1e-9 / t # GB/s

function main()
    if !contains(lowercase(name(device())), "a100")
        @warn("This script was tuned for a NVIDIA A100 GPU. Your GPU: $(name(device())).")
    end
    dtype = Float32
    nthreads = 1024 # CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK
    nblocks = 500_000
    len = nthreads * nblocks # vector length
    a = convert(dtype, 3.1415)
    x = ones(dtype, len)
    y = ones(dtype, len)
    xgpu = CUDA.ones(dtype, len)
    ygpu = CUDA.ones(dtype, len)

    t_broadcast_cpu = @belapsed saxpy_broadcast_cpu!($a, $x, $y) samples = 10 evals = 2
    t_broadcast_gpu = @belapsed saxpy_broadcast_gpu!($a, $xgpu, $ygpu) samples = 10 evals = 2
    t_cuda_kernel = @belapsed saxpy_cuda_kernel!($a, $xgpu, $ygpu; nthreads=$nthreads, nblocks=$nblocks) samples = 10 evals = 2
    t_cublas = @belapsed saxpy_cublas!($a, $xgpu, $ygpu) samples = 10 evals = 2
    times = [t_broadcast_cpu, t_broadcast_gpu, t_cuda_kernel, t_cublas]

    flops = saxpy_flops.(times; len)
    bandwidths = saxpy_bandwidth.(times; dtype, len)

    labels = ["Broadcast (CPU)", "Broadcast (GPU)", "CUDA kernel", "CUBLAS"]
    data = hcat(labels, 1e3 .* times, flops, bandwidths)
    pretty_table(data; header=(["Variant", "Runtime", "FLOPS", "Bandwidth"], ["", "ms", "GFLOP/s", "GB/s"]))
    println("Theoretical Memory Bandwidth: 1555 GB/s")
    return nothing
end

main (generic function with 1 method)

In [74]:
main()

┌─────────────────┬─────────┬─────────┬───────────┐
│         Variant │ Runtime │   FLOPS │ Bandwidth │
│                 │      ms │ GFLOP/s │      GB/s │
├─────────────────┼─────────┼─────────┼───────────┤
│ Broadcast (CPU) │ 202.666 │ 5.05265 │   30.3159 │
│ Broadcast (GPU) │ 5.09245 │ 201.082 │   1206.49 │
│     CUDA kernel │ 4.68801 │  218.43 │   1310.58 │
│          CUBLAS │ 4.67729 │  218.93 │   1313.58 │
└─────────────────┴─────────┴─────────┴───────────┘
Theoretical Memory Bandwidth: 1555 GB/s


<img src="../imgs/a100_saxpy_results.png" width=1000px>

## Remarks

* [KernelAbstractions.jl](https://github.com/JuliaGPU/KernelAbstractions.jl): Writing hardware agnostic computational kernels.
* [Tullio.jl](https://github.com/mcabbott/Tullio.jl): Also supports NVIDIA GPUs and can produce more efficient kernels than simple broadcasting.

More on GPU computing in Julia? See e.g. https://www.youtube.com/watch?v=Hz9IMJuW5hU